In [1]:
library(fixest)
library(tidyr)
library(ggplot2)
library(dplyr)
library(modelsummary)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [25]:
data = read.csv('reg_table_grants.csv')

In [26]:
head(data)

,cntry,dis,pubcnt,domegrant,intgrant,region2
,<chr>,<int>,<int>,<int>,<int>,<chr>
1,Afghanistan,1,14,0,26,Asia
2,Afghanistan,2,7,0,11,Asia
3,Afghanistan,3,2,0,2,Asia
4,Afghanistan,4,4,0,9,Asia
5,Afghanistan,12,1,0,1,Asia
6,Afghanistan,15,3,0,3,Asia


In [27]:
data$domegrant=log(data$domegrant+1)
data$intgrant=log(data$intgrant+1)
data$pubcnt=log(data$pubcnt)

In [30]:
head(data[data$cntry=='United States',])

,cntry,dis,pubcnt,domegrant,intgrant,region2
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>
12981,United States,1,10.074158,10.311416,10.321309,North America
12982,United States,2,10.402261,10.809445,10.698898,North America
12983,United States,3,8.582419,8.738415,8.328451,North America
12984,United States,4,10.320717,11.044792,10.223540,North America
12985,United States,5,9.243872,9.649434,9.003193,North America
12986,United States,6,8.958926,9.776790,9.096724,North America


In [34]:
countries=unique(data$cntry)
fit_regression = function(country, data){
    subset_df = data[data$cntry==country,]
    feols_model = feols(intgrant~pubcnt+domegrant|cntry,data=subset_df)
    return(feols_model)
}
regression_models = lapply(countries, fit_regression, data=data)

In [38]:
unique_df = distinct(data[c('cntry','region2')],.keep_all=TRUE)

In [45]:
regression_models[1]

[[1]]
OLS estimation, Dep. Var.: intgrant
Observations: 39 
Fixed-effects: cntry: 1
Standard-errors: IID 
           Estimate Std. Error  t value  Pr(>|t|)    
pubcnt     1.024061   0.060196 17.01211 < 2.2e-16 ***
domegrant -0.990679   0.543144 -1.82397  0.076466 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.353566     Adj. R2: 0.883333
                 Within R2: 0.889473


In [69]:
extract_estimation=function(reg_model){
    estimation=summary(reg_model)$coeftable['domegrant','Estimate']
    pvalue=summary(reg_model)$coeftable['domegrant','Pr(>|t|)']
    c(estimation,pvalue)
}
estimations = lapply(regression_models, extract_estimation)
estimation_value=sapply(estimations,"[[",1)
pvalue=sapply(estimations,"[[",2)

In [70]:
result_df=data.frame(
cntry=countries,
estimation=estimation_value,
pvalue=pvalue)

In [72]:
result_df=merge(result_df,unique_df,by='cntry')

In [79]:
write.csv(result_df, file='reg_comp_sub_result.csv',row.names=FALSE)